## Step 5 - Transfer learning

Jag valde modellen MobileNetV3 för att göra transfer learning. Delvis för att det är intressant att jämföra min egen modells prestanda med en mindre men mer "professionell" modell. Dels för att träningen och valet av hyperparametrar ska gå snabbare.

In [ ]:
# Running Tensorboard

%load_ext tensorboard
%tensorboard --logdir=./hyperparams

In [ ]:
# Creating image generators
# reusing same structure as on my own model, only changing resolution to 224*224

from keras.preprocessing.image import ImageDataGenerator

image_shape = (224, 224, 3) # Preferred resolution by Mobilenet
test_split_size = 0.15
batch_size = 128
rescale_factor = 1/255

# Define the parameters for data augmentation
train_datagen_2 = ImageDataGenerator(
    rescale=rescale_factor,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


# Create the training dataset
train_dataset_2 = train_datagen_2.flow_from_directory(
    'data/flowers_cleaned_split/train',
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

val_datagen_2 = ImageDataGenerator(rescale=1./255)

# Create the val dataset
val_dataset_2 = val_datagen_2.flow_from_directory(
    'data/flowers_cleaned_split/val/',
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_datagen_2 = ImageDataGenerator(rescale=1./255)

# Create the test dataset
test_dataset_2 = test_datagen_2.flow_from_directory(
    'data/flowers_cleaned_split/test/',
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


print(f'Number Of Images In Training Dataset : {str(train_dataset_2.samples)}')
print(f'Number Of Images In Validation Dataset : {str(val_dataset_2.samples)}')
print(f'Number Of Images In Validation Dataset : {str(test_dataset_2.samples)}')
print(f'Total number of images: {train_dataset_2.samples+val_dataset_2.samples+test_dataset_2.samples}')

In [ ]:
import tensorflow as tf

# Generating tf_datasets from the earlier generated datasets because the hyperband wouldnt accept my datasets
def generator(directory_iterator):
    for x, y in directory_iterator:
        yield x, y

train_dataset_tf = tf.data.Dataset.from_generator(
    lambda: generator(train_dataset_2),
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 224, 224, 3], [None, 3])
)

val_dataset_tf = tf.data.Dataset.from_generator(
    lambda: generator(val_dataset_2),
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 224, 224, 3], [None, 3])
)

In [ ]:
import keras_tuner as kt
from keras.applications import MobileNetV3Large
from keras import Input
from keras.models import Model, load_model
from keras.layers import GlobalAveragePooling2D, Dense, Normalization
from keras.optimizers import Adam

# function to build model, including hyperparameters to be tuned and their respective range
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=1, max_value=3, default=1)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=512)
    learning_rate = hp.Float(
        "learning_rate", min_value=1e-4, max_value=1e-2, default=1e-3, sampling="log"
    )
    

    base_model = MobileNetV3Large(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    inputs = Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)

    for _ in range(n_hidden):
        x = Dense(n_neurons, activation="relu")(x)

    outputs = Dense(5, activation="softmax")(x)

    model = Model(inputs, outputs)

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

# Creating Classificationclass with fit method
class MyClassificationHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)

    def fit(self, hp, model, dataset, **kwargs):
        # Check if normalization is required and the dataset is a TensorFlow dataset
        if hp.Boolean("normalize") and hasattr(dataset, "map"):
            norm_layer = Normalization()
            dataset = dataset.map(lambda x, y: (norm_layer(x), y))
        
        return model.fit(dataset, **kwargs)

# Setting the parameters for the hyperband
hyperband_tuner = kt.Hyperband(
    MyClassificationHyperModel(),
    objective="val_accuracy",
    seed=42,
    max_epochs=10,
    factor=3,
    hyperband_iterations=2,
    overwrite=True,
    directory="hyperparams",
    project_name="hyperband",
)

In [ ]:
from pathlib import Path
from keras.callbacks import TensorBoard, EarlyStopping

# Running the hyperband and logging it to Tensorboard
root_logdir = Path(hyperband_tuner.project_dir) / "tensorboard"
tensorboard_cb = TensorBoard(root_logdir)
early_stopping_cb = EarlyStopping(patience=2)
hyperband_tuner.search(train_dataset_2, epochs=10, validation_data = val_dataset_2,callbacks=[early_stopping_cb, tensorboard_cb] )

In [ ]:
best_model = hyperband_tuner.get_best_models(num_models=1)[0]
best_model.save('hyperparams/my_second_best_model.h5')
# best val_acc = 0.42

In [ ]:
best_hyperparameters = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hyperparameters.values)

# best models parameters:
# n_hidden': 1, 'n_neurons': 425, 'learning_rate': 0.0025890919266874407, 'normalize': True,


In [ ]:
# Training model
steps_per_epoch = train_dataset_2.samples // batch_size
validation_steps = val_dataset_2.samples // batch_size
history = best_model.fit(train_dataset_2,epochs=10,steps_per_epoch = steps_per_epoch,
                    validation_data = val_dataset_2, validation_steps=validation_steps,
                    verbose=True, callbacks=[early_stopping_cb, tensorboard_cb])

## Svar på frågor

- Motivera ditt modellval 
  - Jag valde modellen MobileNetV3 för att göra transfer learning. Delvis för att det är intressant att jämföra min egen modells prestanda med en mindre men mer "professionell" modell. Dels för att träningen och valet av hyperparametrar ska gå snabbare.
- Förklara hur du genomfört transfer learning. 
  - Jag skapade en base_model där jag valda att inte inkludera det slutgiltiga Denselagrerna, jag valde också att göra de inkluderade lagrerna ej träningsbara.
- Förklara hur du systematiskt valt de bästa hyperparametrarna (ska framgå i koden)
  - Jag använde Hyperband som är en slags randomiserad "turnering" mellan olika värden. Variablerna jag valda att randomisera är:
    - Antal lager, mellan 1-3
    - Antalet neuroner per lager, mellan 16 och 512
    - Learning rate, mellan 0.01 och 0.0001
    - Normalisering, True eller False
  - Den bästa modellen blev inte särskilt bra, men fick parametrarna: n_hidden': 1, 'n_neurons': 425, 'learning_rate': 0.0025890919266874407, 'normalize': True